### 주차별 프로젝트(1주차 : 종이 헬리콥터 회귀분석)

구성 변수 : 날개 길이, 날개 폭, 다리 길이, 몸통 길이 + ɑ

모델 구성 : scikit-learn, NN

### 데이터 구성
실제로 만들어서 시간을 관측하고, 시간을 측정하는 기준은 아래와 같다.

- 두 날개의 가장 안쪽을 두 손가락을 이용해 천장에 붙인 뒤 낙하

In [1]:
import pandas as pd

dataset = {
    "wing_length" : [11.0, 9.5, 10.2, 16.0, 9.5, 13.25, 13.25, 11.8, 8.5, 8.0, 9.7, 9.0, 9.0, 9.0, 7.5],
    "wing_width" : [6.25, 7.4, 5.3, 7.4, 7.4, 7.45, 7.45, 6.95, 2.7, 2.7, 3.0, 3.0, 3.0, 3.0, 2.7],
    "body_length" : [3.0, 3.0, 1.8, 1.0, 3.0, 2.0, 2.0, 2.25, 2.0, 2.0, 3.0, 3.0, 2.5, 2.15, 1.65],
    "leg_length" : [7.05, 8.5, 2.5, 4.0, 3.6, 4.65, 3.8, 4.8, 2.5, 2.5, 3.5, 3.5, 4.0, 4.3, 2.8],
    "target" : [2.31, 1.77, 2.73, 2.43, 2.27, 2.5, 2.28, 2.41, 2.41, 2.76, 2.98, 2.69, 2.73, 2.8, 2.83]
}

len(dataset['wing_length'])
df = pd.DataFrame(dataset, columns = dataset.keys(), index = list(range(len(dataset['wing_length']))))
df.shape

(15, 5)

### 학습 데이터 구성
- target값과 데이터 분할, 정규화 진행

In [2]:
X = df.drop(labels = 'target', axis = 1)
y = df[['target']]

In [3]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0, 1))
X = scaler.fit_transform(X)
X

array([[0.41176471, 0.74736842, 1.        , 0.75833333],
       [0.23529412, 0.98947368, 1.        , 1.        ],
       [0.31764706, 0.54736842, 0.4       , 0.        ],
       [1.        , 0.98947368, 0.        , 0.25      ],
       [0.23529412, 0.98947368, 1.        , 0.18333333],
       [0.67647059, 1.        , 0.5       , 0.35833333],
       [0.67647059, 1.        , 0.5       , 0.21666667],
       [0.50588235, 0.89473684, 0.625     , 0.38333333],
       [0.11764706, 0.        , 0.5       , 0.        ],
       [0.05882353, 0.        , 0.5       , 0.        ],
       [0.25882353, 0.06315789, 1.        , 0.16666667],
       [0.17647059, 0.06315789, 1.        , 0.16666667],
       [0.17647059, 0.06315789, 0.75      , 0.25      ],
       [0.17647059, 0.06315789, 0.575     , 0.3       ],
       [0.        , 0.        , 0.325     , 0.05      ]])

In [4]:
y = scaler.fit_transform(y)
y

array([[0.44628099],
       [0.        ],
       [0.79338843],
       [0.54545455],
       [0.41322314],
       [0.60330579],
       [0.4214876 ],
       [0.52892562],
       [0.52892562],
       [0.81818182],
       [1.        ],
       [0.76033058],
       [0.79338843],
       [0.85123967],
       [0.87603306]])

### 모델 빌드1 : ANN
- input layer's shape = (4, ) ➡️ 4개의 독립변수
- 64개의 노드를 가지는 hidden layer 4개층 구성, 활성화 함수는 relu 사용
- output layer's shape = (1, ) ➡️ 비행시간에 해당하는 1개의 출력 변수, 항등함수로 활성화

### 모델 빌드 2 : scikit learn
다중 선형회귀 분석 모델들로 시도

#### 신경망 모델링

In [5]:
from keras.models import Model
from keras.layers import Dense, Input
from keras.optimizers import Adam

input_layer = Input(shape = (4, ))
x = Dense(64, activation = "relu")(input_layer)
x = Dense(64, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
output_layer = Dense(1, activation = "sigmoid")(x)

optim = Adam(learning_rate = 0.001)

model = Model(input_layer, output_layer)
model.compile(optimizer = optim, loss = "mean_squared_error")

In [ ]:
model.fit(X, y, epochs = 500)

# clear output of training cell

### 예측값 산출
- 별도의 테스트 데이터가 없기에, 훈련 시에 사용한 데이터를 기반으로 예측

In [7]:
predicted_list = []
for i in range(X.shape[0]):
    predicted_list.append(model.predict(X)[i][0])

1/1 [==============================] - 0s 15ms/step


### 기존 데이터 프레임과 병합
- 기존 데이터를 복사하여 예측된 값들과 함께 하나의 데이터프레임으로 병합한다.

In [8]:
import numpy as np

predicted_list = np.array(predicted_list).reshape(-1, 1)
predicted_list = scaler.inverse_transform(predicted_list)

df1 = df.copy()
df1['predicted_target'] = predicted_list

In [9]:
df1

,wing_length,wing_width,body_length,leg_length,target,predicted_target
0,11.00,6.25,3.00,7.05,2.31,2.282312
1,9.50,7.40,3.00,8.50,1.77,1.773568
2,10.20,5.30,1.80,2.50,2.73,2.723040
3,16.00,7.40,1.00,4.00,2.43,2.417215
4,9.50,7.40,3.00,3.60,2.27,2.247437
5,13.25,7.45,2.00,4.65,2.50,2.484757
6,13.25,7.45,2.00,3.80,2.28,2.264823
7,11.80,6.95,2.25,4.80,2.41,2.388357
8,8.50,2.70,2.00,2.50,2.41,2.399604
9,8.00,2.70,2.00,2.50,2.76,2.750018


### svr 모델로 회귀 모델 생성

In [10]:
from sklearn.svm import LinearSVR
from sklearn.model_selection import GridSearchCV

lin = LinearSVR(random_state = 22)

### GridSearchCV를 활용한 하이퍼 파라미터 튜닝

In [11]:
params = {
    'epsilon' : np.logspace(-2, 2, 5),
    'C' : np.logspace(-3, 0, 4),
    'max_iter' : list(range(1000, 3501, 500))
}

lin_grid = GridSearchCV(lin, param_grid = params, n_jobs = -1, cv = 5)
y = y.reshape(-1)
lin_grid.fit(X, y)

GridSearchCV(cv=5, estimator=LinearSVR(random_state=22), n_jobs=-1,
             param_grid={'C': array([0.001, 0.01 , 0.1  , 1.   ]),
                         'epsilon': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'max_iter': [1000, 1500, 2000, 2500, 3000, 3500]})

In [12]:
lin_grid.best_params_

{'C': 1.0, 'epsilon': 0.01, 'max_iter': 1000}

In [13]:
from sklearn.metrics import mean_squared_error

y_pred = lin_grid.best_estimator_.predict(X)
mean_squared_error(y, y_pred) # 0.018의 mse 지표를 확인, 타 모델들과 비교해볼 것

0.017752870619342654